In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_confusion_matrix,precision_score,recall_score,f1_score,accuracy_score,classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 
sns.set_style("darkgrid")

Loading data

In [ ]:
food_data=pd.read_csv('/kaggle/input/indian-food-101/indian_food.csv')
food_data.head()

# EDA

In [ ]:
food_data.shape

This dataset has 255 rows and it gives detailed information about food name, ingredients,diet,rep_time,cook_time,flavor_profile,course,state and region in INDIA

In [ ]:
food_data.isnull().sum()

In [ ]:
#Dropping NA
food_data.dropna(inplace=True)

In [ ]:
food_data.dtypes

In [ ]:
food_data.describe()

In this dataset, many data are presented as -1.

# Column wise treating -1 value

* Numerical column****

In [ ]:
food_data[food_data['prep_time']==-1].count()

In [ ]:
food_data['prep_time']=food_data.prep_time.replace(-1,food_data['prep_time'].mean())

In [ ]:
food_data[food_data['cook_time']==-1].count()

In [ ]:
food_data['cook_time']=food_data.prep_time.replace(-1,food_data['cook_time'].mean())

* ****Categorical column

In [ ]:
food_data[food_data['region']=='-1'].count()

In [ ]:
food_data['region']=food_data.region.replace('-1',None)

In [ ]:
food_data[food_data['state']=='-1'].count()

In [ ]:
food_data['state']=food_data.state.replace('-1',None)

In [ ]:
food_data[food_data['flavor_profile']=='-1'].count()

In [ ]:
food_data['flavor_profile']=food_data.flavor_profile.replace('-1',None)

# Adding columns for better understanding

In [ ]:
c=0
def split_ingredients(col):
    c=col.split(',')
    return c

In [ ]:
l=[]
for i in food_data['ingredients']:
    s=0
    for j in split_ingredients(i):
        s+=1
    l.append(s)
food_data['ingredients_count']=l

In [ ]:
food_data.head()

# Data Distribution

* ****Univarient Analysis

In [ ]:
sns.distplot(food_data['prep_time'],color='green');
plt.title("Preparation Time",fontsize='xx-large');

<p>Preparation column data is right skewed </p>

In [ ]:
sns.distplot(food_data['cook_time'],color='blue');
plt.title("Cooking Time",fontsize='xx-large');

<p>Cooking Time data is right skewed </p>

In [ ]:
sns.pairplot(food_data);

* ****Bivarient Analysis

In [ ]:
plt.figure(figsize=(15,7))
sns.countplot(y=food_data['state']);
plt.title("State wise food items count",color='black',fontsize='xx-large');

Compared to other states, Punjab has high number of food items.

In [ ]:
plt.figure(figsize=(10,4))
sns.countplot(food_data['course']);
plt.title("Courses CountPlot",color='black',fontsize='xx-large');

This data covers mostly main course dishes. 

In [ ]:
plt.figure(figsize=(10,4))
sns.barplot(food_data['course'],food_data['prep_time'],hue=food_data['flavor_profile'],palette='magma');
plt.title("Impact of flavors in course and preparation time",color='black',fontsize='xx-large');

Comparatively the spicy starters are famous in many states and it take more time to prepare also.

In [ ]:
plt.figure(figsize=(10,4))
sns.barplot(food_data['course'],food_data['cook_time'],hue=food_data['diet']);
plt.title("Impact of diet in course and cook time",color='black',fontsize='xx-large');

Non-vegetarian diets are preferred in many states and it takes more time to cook also. 

In [ ]:
plt.figure(figsize=(10,4))
sns.countplot(food_data['flavor_profile'],palette='magma');
plt.title("Flavours CountPlot",color='black',fontsize='xx-large');

* **** Heat Map

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(food_data.corr(),annot=True,cmap=plt.cm.Accent)

In [ ]:
plt.figure(figsize=(15,8))
plt.title("Flavours Domination",color='black',fontsize='xx-large');
plt.pie(food_data['flavor_profile'].value_counts().reset_index()['flavor_profile'],labels=food_data['flavor_profile'].value_counts().reset_index()['flavor_profile'], explode=None, colors=None, autopct=None, shadow=False);
plt.legend(food_data['flavor_profile'].value_counts().reset_index()['index'],loc="best");

It seems across all states, people are very fond of Spicy & Sweets foods than sour and bitter. 

In [ ]:
plt.figure(figsize=(20,10))
plt.title("Segmentation of vegetarian and Non-vegetarian",color='black',fontsize='xx-large');
plt.pie(food_data['diet'].value_counts().reset_index()['diet'],labels=food_data['diet'].value_counts().reset_index()['index'], explode=None, colors=None, autopct=None, shadow=False);
plt.legend(food_data['diet'].value_counts().reset_index()['index'],loc="best");

People are most common to vegetarian diet than non-vegetarian.

In [ ]:
plt.figure(figsize=(20,10))
plt.title("Segmentation of regions",color='black',fontsize='xx-large');
plt.pie(food_data['region'].value_counts().reset_index()['region'],labels=food_data['region'].value_counts().reset_index()['index'], explode=None, colors=None, autopct=None, shadow=False);
plt.legend(food_data['region'].value_counts().reset_index()['index'],loc="best");

# Total ingredients used all over states

In [ ]:
wordCloud = WordCloud(background_color='black',max_font_size = 50).generate(' '.join(food_data['name']))
plt.figure(figsize=(20,7))
plt.axis('off')
plt.title('Total Ingredients Used',color='black',fontsize='xx-large');
plt.imshow(wordCloud)
plt.show()

# Curious about Non-vegetarian & vegetarian ingredients

In [ ]:
veg=food_data.where(food_data['diet']=='vegetarian')
veg=veg.dropna()
veg.shape

In [ ]:
nonveg=food_data.where(food_data['diet']=='non vegetarian')
nonveg=nonveg.dropna()
nonveg.shape

In [ ]:
wordCloud = WordCloud(background_color='skyblue',max_font_size = 50).generate(' '.join(veg['name']))
plt.figure(figsize=(20,7))
plt.axis('off')
plt.title('Vegetarion Diet Ingredients',color='black',fontsize='xx-large');
plt.imshow(wordCloud)
plt.show()

In [ ]:
wordCloud = WordCloud(background_color='violet',max_font_size = 50).generate(' '.join(nonveg['name']))
plt.figure(figsize=(20,7))
plt.axis('off')
plt.title('Non-Vegetarion Diet Ingredients',color='black',fontsize='xx-large');
plt.imshow(wordCloud)
plt.show()

# Ingredients state wise usuage

In [ ]:
plt.figure(figsize=(35,10))
sns.barplot(x=food_data['state'],y=food_data['ingredients_count']);
plt.title("State wise food items count",color='black',fontsize='xx-large');

# Top 10 Ingredients

In [ ]:
total_ingredients = pd.Series(food_data.ingredients.str.split(',').sum()).value_counts()

In [ ]:
plt.figure(figsize=(20,10))
top_ingredients = total_ingredients[total_ingredients>12]
plt.title("Top 10 Ingredients",color='black',fontsize='xx-large');
plt.pie(top_ingredients.values,labels=top_ingredients.index, explode=None, colors=None, autopct=None, shadow=False);

# Food without these ingredients is not possible !

In [ ]:
total_ingredients = total_ingredients[total_ingredients>22]
plt.title('Most used ingredients',color='black',fontsize='xx-large')
plt.bar(total_ingredients.index,total_ingredients.values,color=('Yellow', 'red', 'green', 'blue', 'cyan'));

# Conclusion

<p> All states dishes are not prepared without these top 5 ingrdients like sugar, ginger, grammasala, curry leaves, ghee <p>